# Automate Inference with the AWS Step Functions Data Science SDK

## Install SDK

In [ ]:
!pip install --upgrade -q stepfunctions

## Setup environment

In [ ]:
import json
import uuid
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
import stepfunctions
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow, cloudformation
from stepfunctions.steps import Chain, ProcessingStep, TransformStep, Catch, Fail, Succeed


sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() # execution role for SageMaker
workflow_execution_role = role # execution role for Step Functions
bucket = sagemaker_session.default_bucket() # you can specify a bucket name here

## Execution input placeholders

In [ ]:
job_name = uuid.uuid1().hex
processing_input = '<your-data-s3-path>' 
processing_output= f's3://{bucket}/{job_name}/data/processed/'
transform_input = f's3://{bucket}/{job_name}/data/processed/test_batch_transform.csv'
transform_output = f's3://{bucket}/{job_name}/data/predicted/'
model_artefact = '<your-model-s3-path>'

execution_input = ExecutionInput(
    schema={
        "JobName": str,
        "Processing": {
            "Input": str,
            "Output": str
        }
    }
)

## Process data

In [ ]:
data_processor = Processor(
    role=role, 
    image_uri='<your-container-image-uri>', 
    instance_count=1, 
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=30, 
    max_runtime_in_seconds=1200
)

In [ ]:
input_folder = '/opt/ml/processing/input'
output_folder = '/opt/ml/processing/output'

inputs = [
    ProcessingInput(
        input_name='input',
        source=execution_input["Processing"]["Input"],
        destination=input_folder
    )
]

outputs = [
    ProcessingOutput(
        output_name='preprocessed',
        source=output_folder,
        destination=execution_input["Processing"]["Output"]
    )
]

## Create SageMaker model and Transformer

In [ ]:
model = PyTorchModel(model_data=model_artefact,
                   name=name_from_base('bert-model'),
                   role=role, 
                   entry_point='predict_batch.py',
                   source_dir='source_dir',
                   framework_version='1.5.0')

transformer = model.transformer(
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    strategy='SingleRecord',
    assemble_with='Line',
    accept = 'text/csv',
    max_concurrent_transforms=50,
    output_path=transform_output
)

## Define workflow steps

In [ ]:
processing_step = ProcessingStep(
    state_id="Process Data",
    processor=data_processor,
    job_name=execution_input["JobName"],
    inputs=inputs,
    outputs=outputs,
    container_arguments=[f"--input={input_folder}", f"--output={output_folder}"],
    result_path="$.Processing"
)

In [ ]:
transformer_step = TransformStep(
    state_id='Predict Batch',
    transformer=transformer,
    job_name=execution_input['JobName'],     
    model_name=model.name, 
    data=transform_input,
    content_type='text/csv',
    split_type='Line',
    join_source='Input',
    result_path="$.Inference"
)

In [ ]:
# error catching, failure and success steps
failed = Fail(state_id="Failed")
succeed = Succeed(state_id="Succeed")

catch_failures = Catch(error_equals=["States.ALL"], next_step=failed)
processing_step.add_catch(catch_failures)
transformer_step.add_catch(catch_failures)

## Create workflow pipeline

In [ ]:
workflow_graph = Chain([
        processing_step,
        transformer_step,
        succeed
])

workflow_pipeline = Workflow(
    name="BatchWorkflow",
    definition=workflow_graph,
    execution_input=execution_input,
    role=workflow_execution_role
)

In [ ]:
print(workflow_pipeline.definition.to_json(pretty=True))

In [ ]:
workflow_pipeline.render_graph(portrait=True)

## Create/update state machine and execute

In [ ]:
workflow_pipeline.create()
# workflow_pipeline.update(workflow_pipeline_definition)

## Workflow pipeline inputs

In [ ]:
execution_inputs = {
    "JobName": job_name, 
    "Processing": {
        "Input": processing_input, 
        "Output": processing_output
    }
}

workflow_pipeline.execute(inputs=execution_inputs)

## Generate CloudFormation template

In [ ]:
print(workflow_pipeline.get_cloudformation_template())